# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,37.15,70,20,12.10,GS,1706033786
1,1,ponta delgada,37.7333,-25.6667,65.19,77,75,16.11,PT,1706033864
2,2,nuku'alofa,-21.1333,-175.2000,79.14,94,75,5.75,TO,1706033865
3,3,belchatow,51.3688,19.3567,39.97,91,98,14.58,PL,1706033865
4,4,tura,25.5198,90.2201,54.32,68,1,2.30,IN,1706033866


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
hover_columns = ['City', 'Humidity']
plot = city_data_df.hvplot.points('Lng', 'Lat', geo=True, tiles='OSM', size=city_data_df['Humidity'], 
                                  width=800, height=550, c='Humidity', cmap='spectral', alpha=0.5, hover_cols=hover_columns)

# Display the map
plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,_size,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[(city_data_df['Max Temp'] < 75) & (city_data_df['Max Temp'] > 62) & (city_data_df['Humidity'] < 40)]

# Drop any rows with null values
ideal_weather_df.dropna(inplace=True)

# Display sample data
ideal_weather_df

C:\Users\K\AppData\Local\Temp\ipykernel_14468\468422291.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_weather_df.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,45,kuraymah,18.5500,31.8500,73.02,19,1,12.30,SD,1706033882
119,119,as sulayyil,20.4607,45.5779,70.05,31,32,9.55,SA,1706033915
154,154,chonchi,-42.6128,-73.8092,70.81,34,14,1.95,CL,1706033931
203,203,merowe,18.4833,31.8167,73.09,19,2,12.30,SD,1706033952
266,266,smara,26.7384,-11.6719,73.98,17,94,18.61,EH,1706033981
268,268,ad dilam,23.9915,47.1627,70.21,29,41,10.89,SA,1706033982
272,272,ad dabbah,18.0500,30.9500,72.54,21,1,11.86,SD,1706033983
281,281,tamanrasset,22.7850,5.5228,62.51,25,0,12.66,DZ,1706033987
345,345,umm kaddadah,13.6017,26.6876,71.13,12,76,12.62,SD,1706034018
356,356,al mishkhab,31.8042,44.4911,64.51,27,100,15.88,IQ,1706034023


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
45,kuraymah,SD,18.5500,31.8500,19,
119,as sulayyil,SA,20.4607,45.5779,31,
154,chonchi,CL,-42.6128,-73.8092,34,
203,merowe,SD,18.4833,31.8167,19,
266,smara,EH,26.7384,-11.6719,17,
268,ad dilam,SA,23.9915,47.1627,29,
272,ad dabbah,SD,18.0500,30.9500,21,
281,tamanrasset,DZ,22.7850,5.5228,25,
345,umm kaddadah,SD,13.6017,26.6876,12,
356,al mishkhab,IQ,31.8042,44.4911,27,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'limit': 1,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter=circle"] = f'{lng},{lat},{radius}'
    params["bias=proximity"] = f'{lng},{lat}'
    params['apiKey'] = geoapify_key
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty
    name_response = requests.get(f'{base_url}{params}')
    
    # Convert the API response to JSON format
    name_address = name_response.json()
    pprint(name_response)
    break
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
# hotel_df

Starting hotel search
<Response [401]>


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE